# Datenvorbereitung: Vektorisierung unstrukturierter Daten und Feature Engineering

Bisher hast du dich mit strukturierten Daten beschäftigt, wo du häufig die (numerischen) Werte direkt als Features nutzen konntest. In diesem Teil wirst du unstrukturierte Daten betrachten und dich dabei speziell auf Texte konzentrieren. Hier liegt der Fall nicht ganz so einfach, weil du zunächst Features finden musst.

## Texte vektorisieren

Texte sind ein Spezialfall, weil sie sich relativ einfach in Vektorform überführen lassen und es ein Standardverfahren dafür gibt, das sich für einfache Textanalyse sehr gut bewährt hat.

Willst du mehrere Dokumente *vektorisieren*, stellst du dir dazu zunächst das Vokabular auf und nummerierst es durch. Deine Vektoren sind also so groß wie das Vokabular, das kann ziemlich viel Wörter umfassen. 

Für jedes Dokument berechnest du dir anschließend, wie häufig die Wörter darin vorkommen und setzt diese Anzahl an die entsprechende Vektorposition. Ein solcher Dokumentvektor besteht zu erheblichen Teilen aus `0`, er ist also dünn besetzt. Das gleiche gilt für die Menge aller Dokumentvektoren, die sog. *Dokumenten-Term-Matrix*.

`scikit-learn` unterstützt dich dabei sehr gut. Zunächst kannst du mit einem eingebauten Datenset arbeiten, das Posts aus 20 Newsgroups (aus der Anfang des Internets bzw. Usenets) enthält:

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups()

Die Textdaten findest du im Feld `data` des zurückgelieferten `dict`, die jeweilige Newsgroup in `target`. Am besten konvertierst du das in einen `DataFrame`:

In [2]:
import pandas as pd
newsgroups_df = pd.DataFrame(newsgroups.data, columns=["text"])
newsgroups_df["newsgroup"] = [newsgroups.target_names[t] for t in newsgroups.target]
newsgroups_df

,text,newsgroup
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,sci.space
...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,comp.graphics


Und jetzt kannst du fast schon mit der Vektorisierung Learning beginnen. Dabei werden die Texte in einzelne Wörter unterteilt (tokenisiert) und anschließend die Wörter durchnummeriert. Für jedes Dokument wird gezählt, welches Wort wie häufig darin vorkommt. So entstehen die Dokumentvektoren:

### CountVectorizer

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cvectors = cv.fit_transform(newsgroups_df["text"])

Nun kannst du dir anschauen, wie das entstehende Objekt aussieht:

In [4]:
cvectors

<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

Hier kannst du erkennen, dass es sich um eine *sehr große Matrix* handelt. In den 11.314 Posts sind über 130.000 unterschiedliche Wörter aufgetaucht. Zum Glück kommen in den meisten Dokumente nur sehr wenige Wörter vor, so dass die Matrix [dünn besetzt](https://de.wikipedia.org/wiki/D%C3%BCnnbesetzte_Matrix) ist. Diese sog. Sparse Matrices können sehr effizient dargestellt werden:

In [5]:
cvectors.data.nbytes

14300520

Die gesamte Matrix braucht nicht mal 1,5 MB RAM. In einer naiven Darstellung wären das hingegen `11314 * 130107 * 4 Bytes = 5888122392 Bytes`, was fast 6 GB entspricht. 

Eine nette Anwendung des `CountVectorizers` ist die Bestimmung der häufigsten Wörter im gesamten Datenset. Dazu summierst du die Werte einfach auf:

In [6]:
freq = cvectors.sum(axis=0).A[0]
freq

array([1534,  953,    7, ...,    1,    2,    3])

Die Zahlen entsprechen dabei den Anzahlen der Wörter im Vokabular. Die häufigsten 10 kannst du so ermitteln (das `[::-1]` dient zur Umkehrung der Sortierreihenfolge, so dass die häufigsten am Anfang stehen): 

In [7]:
top10 = freq.argsort()[::-1][0:10]
top10

array([114455, 115475,  89362,  30827,  28146,  66608,  68532, 114440,
        68766,  56283])

Nun kannst du das Vokabular nutzen, um dir die Wörter ausgeben zu lassen:

In [8]:
voc = cv.get_feature_names()
{ voc[pos]: freq[pos] for pos in top10 }

/home/cwinkler/.venv/jupyter-new/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


{'the': 146532,
 'to': 75064,
 'of': 69034,
 'ax': 62406,
 'and': 57957,
 'in': 49401,
 'is': 43480,
 'that': 39264,
 'it': 33638,
 'for': 28600}

Wie du siehst, haben die häufigsten Wörter fast keine Bedeutung und werden normalerweise als sog. *Stoppwörter* weggelassen.

### Feature-Engineering mit `CountVectorizer`

Dennoch können solche umfangreichen Vektoren unhandlich sein. Du kannst die Dimension reduzieren, indem du z.B. nur Wörter berücksichtigst, die in mindestens 5 Dokumenten vorkommen:

In [9]:
cv = CountVectorizer(min_df=5)
cvectors = cv.fit_transform(newsgroups_df["text"])
cvectors

<11314x25941 sparse matrix of type '<class 'numpy.int64'>'
	with 1635098 stored elements in Compressed Sparse Row format>

Der `CountVectorizer` hat viele Optionen, mit denen die Dimension beeinflusst werden kann. So kannst du etwa die Dimension explizit vorgeben, aber auch die Maximalanzahl der Dokumente vorgeben, in denen Wörter vorkommen dürfen.

Schließlich kannst du auch sog. [N-Gramme](https://de.wikipedia.org/wiki/N-Gramm) nutzen, mit denen du Wortkombinationen berücksichtigst. Achtung, dadurch wächst die Dimension beträchtlich:

In [10]:
cvn = CountVectorizer(ngram_range=(1,2), min_df=5)
cvectorsn = cvn.fit_transform(newsgroups_df["text"])
cvectorsn

<11314x112027 sparse matrix of type '<class 'numpy.int64'>'
	with 3212582 stored elements in Compressed Sparse Row format>

Um Wörter weniger stark zu gewichten, die ein geringes Differenzierungspotenzial haben, kannst du das sog. [TF/IDF-Maß](https://de.wikipedia.org/wiki/Tf-idf-Ma%C3%9F) verwenden. `scikit-learn` kann diese Transformation direkt für dich durchführen:

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tvectors = tfidf.fit_transform(cvectors)
tvectors

<11314x25941 sparse matrix of type '<class 'numpy.float64'>'
	with 1635098 stored elements in Compressed Sparse Row format>

Die Dimensionen ändern sich dadurch nicht, aber für viele Machine Learning-Verfahren hast du damit eine bessere Darstellung der Dokumentvektoren gefunden.

## Sprache vektorisieren

Sprache lässt sich normalerweise nicht direkt vektorisieren. In den meisten Fällen wirst du ein sog. [Speech-to-Text-Verfahren](https://de.wikipedia.org/wiki/Spracherkennung) verwenden, um die gesprochene Sprache in Text zu wandeln. Anschließend kannst du die oben erklärten Methoden zur Textanalyse verwenden.

Leider gehen bei dieser Methode Informationen verloren, die du nicht wieder rekonstruieren kannst. So bleiben etwa die *Emotionen* auf der Strecke, die durch Betonung etc. vermittelt werden. Wenn du solche Anforderungen hast, gibt es wieder spezielle Algorithmen, die dies extrahieren können. Dieses Gebiet ist allerdings noch ziemlich neu, so dass wir uns im Rahmen des Kurses damit nicht beschäftigen werden.

## Bilder vektorisieren

Bilder sind deutlich schwieriger zu verarbeiten und zu vektorisieren. Zunächst werden die Bilder auf eine Einheitsgröße von z.B. 100x100 Pixel und Graustufen reduziert. Die daraus entstehenden 10.000 Werte repräsentieren dann ein Bild.

Anders als bei Texten ist der unmittelbare Zusammenhang der Vektoren zu den Bildern (bzw. zu dem, was auf den Bilder dargestellt wird) nicht mehr gegeben. Daher wird für Machine Learning mit Bildern fast immer Deep Learning benutzt, damit im ersten Layer des Netzwerks durch das Training im Prinzip bessere Vektoren für die Bilder konstruiert werden können.

Eine häufige Fragestellung bei Bilder ist die sog. *Object Detection*, d.h. du möchtest gerne wissen, was sich auf den Bilder befindet. Das Training dafür benötigt sehr große Datenmengen, d.h. sehr viele Bilder, bei denen die abgebildeten Objekte schon bekannt sind. Häufig gibt es dazu bereits vortrainierte Modelle, die dur direkt nutzen kannst. Diese übernehmen dann auch die Konvertierung der Bilder für dich.

Beispiele für *Object Detection* findest du bei [YoloV5](https://github.com/ultralytics/yolov5), das ein sehr schnelles Modell bereitstellt. Möchtest du auch Konturen erkennen, kannst du z.B. [Detectron2](https://github.com/facebookresearch/detectron2) von Facebook Research verwenden.

## Videos vektorisieren

Für Videos gelten ähnlich Bedingungen wie für Bilder - auch hier kannst du Objekte erkennen und diese dann verwenden.

Erschwerend kommt hier allerdings dazu, dass die Objekte sich im Laufe der Zeit ändern, also eine weitere Dimension. Das kann helfen, aber auch zur deutliche Komplizierung des Vorgehens führen.

Die meisten Videos sind außerdem mit Sprache versehen, die du mit den oben geschilderten Verfahren zunächst in Text wandeln solltest.

## Vektorisierung nicht unterschätzen

Ob du ein Machine Learning-Projekt erfolgreich abschließen kannst, liegt zu einem erheblichen Teil an der korrekten Vektorisierung. Dabei geht es vor allem auch darum, die richtigen *Features* zu finden.

* Bei strukturierten Daten hast du es dabei am einfachsten - du kannst direkt mit Zahlen arbeiten, die du evtl. noch skalieren musst.
* Bei Texten ist es etwas schwieriger, häufig kannst du mit der Document-Term-Matrix arbeiten und Wörter (oder Bigramme) als Features verwenden.
* Sprache wandelst du am besten in Text.
* Bei Bildern kannst du häufig die dort abgebildeten Objekte als Features verwenden.
* Videos kombinieren Bilder mit Zeitabhängigkeit sowie Sprache und stellen daher eine echte Herausforderung dar.

Dieses sog. *Feature Engineering* wird dir in vielen Projekten begegnen und ist in seiner Wichtigkeit kaum zu unterschätzen.